In [1]:
import pandas as pd
df_spo_wiki=pd.read_csv('spo_wiki.csv')
df_spo_wiki.info()
df_spo_wiki.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2920 entries, 0 to 2919
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   subjectLabel    2920 non-null   object
 1   predicateLabel  2920 non-null   object
 2   objectLabel     2920 non-null   object
 3   subject         2920 non-null   object
 4   object          2920 non-null   object
 5   source          2920 non-null   object
dtypes: object(6)
memory usage: 137.0+ KB


,subjectLabel,predicateLabel,objectLabel,subject,object,source
0,pearl,instance of,spherical body,Q43436,Q112511193,Q83437
1,diamond,made from material,carbon,Q5283,Q623,Q83437
2,ruby,color,red,Q43088,Q3142,Q83437
3,pearl,made from material,nacre,Q43436,Q215865,Q83437
4,ruby,described by source,Brockhaus and Efron Encyclopedic Dictionary,Q43088,Q602358,Q83437


In [2]:
df=df_spo_wiki
df_pivot = pd.pivot_table(
    df.reset_index(),
    # columns=['col1'],
    index=['predicateLabel'],
    values=['index'], 
    aggfunc={
        'index': ["count"],
    },
    fill_value=0
)
df_pivot.columns = ['_'.join(col).strip() for col in df_pivot.columns.values]
df_pivot.sort_values('index_count',ascending=False).head(50)

,index_count
predicateLabel,
subclass of,900
described by source,356
instance of,259
named after,252
IMA status and/or rank,187
crystal system,158
type locality (geology),134
space group,74
country of origin,62


In [3]:
values_to_drop = {
    'described by source',
    'different from',
    "topic's main category",
    'on focus list of Wikimedia project',
    "topic's main template",
    'maintained by WikiProject'
}
df2 = df[~df['predicateLabel'].isin(values_to_drop)]
df2.info()
df2.head()

<class 'pandas.core.frame.DataFrame'>
Index: 2441 entries, 0 to 2919
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   subjectLabel    2441 non-null   object
 1   predicateLabel  2441 non-null   object
 2   objectLabel     2441 non-null   object
 3   subject         2441 non-null   object
 4   object          2441 non-null   object
 5   source          2441 non-null   object
dtypes: object(6)
memory usage: 133.5+ KB


,subjectLabel,predicateLabel,objectLabel,subject,object,source
0,pearl,instance of,spherical body,Q43436,Q112511193,Q83437
1,diamond,made from material,carbon,Q5283,Q623,Q83437
2,ruby,color,red,Q43088,Q3142,Q83437
3,pearl,made from material,nacre,Q43436,Q215865,Q83437
5,ruby,streak color,white,Q43088,Q23444,Q83437


In [4]:
import json
def spo2kg(df_spo_wiki):
    df = df_spo_wiki
    df_spo = df_spo_wiki[['subjectLabel','predicateLabel','objectLabel']]
    
    metadata = {}
    # Base Wikipedia URL for metadata
    wiki_base_url = "https://en.wikipedia.org/wiki/"
    
    for i in df.itertuples():
        # print(i.subjectLabel)
        
        # Add subject metadata (Wikidata ID and Wikipedia URL)
        if i.subjectLabel not in metadata:
            metadata[i.subjectLabel] = {
                "url": f"{wiki_base_url}{i.subjectLabel.replace(' ', '_')}",
                "wiki_id": i.subject
            }
        
        # Add object metadata (Wikidata ID and Wikipedia URL)
        if i.objectLabel not in metadata:
            metadata[i.objectLabel] = {
                "url": f"{wiki_base_url}{i.objectLabel.replace(' ', '_')}",
                "wiki_id": i.object
            }
    
    return df_spo, metadata

df_spo, metadata = spo2kg(df2)
df_spo.info()
df_spo.head()

<class 'pandas.core.frame.DataFrame'>
Index: 2441 entries, 0 to 2919
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   subjectLabel    2441 non-null   object
 1   predicateLabel  2441 non-null   object
 2   objectLabel     2441 non-null   object
dtypes: object(3)
memory usage: 76.3+ KB


,subjectLabel,predicateLabel,objectLabel
0,pearl,instance of,spherical body
1,diamond,made from material,carbon
2,ruby,color,red
3,pearl,made from material,nacre
5,ruby,streak color,white


In [5]:
csv_file= 'data_drop.csv'
metadata_file='metadata_drop.json'

df_spo.to_csv(csv_file, header=False, index=False)
with open(metadata_file, 'w', encoding='utf-8') as f:
    json.dump(metadata, f, indent=2)

In [6]:
!kg add -f data_drop.csv
!kg meta -f metadata_drop.json
!kg start

🎉 Starting the app.
 * Serving Flask app 'kgsearch.app.app' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [02/Oct/2024 19:21:42] "GET /search/1/1/1/ruby HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2024 19:21:44] "GET /search/1/1/1/ruby; HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2024 19:21:47] "GET /search/1/1/1/ruby;topaz HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2024 19:21:48] "GET /search/1/1/1/ruby;topaz; HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2024 19:21:55] "GET /search/1/1/1/ruby;topaz;Sapphire HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2024 19:22:00] "GET /search/1/1/1/ruby;topaz;Sapphire; HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2024 19:22:02] "GET /search/1/1/1/ruby;topaz;Sapphire;Aquamarine HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2024 19:22:22] "GET /search/1/1/1/ruby;topaz;Sapphire;beryl HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2024 19:22:23] "GET /search/1/1/1/ruby;topaz;Sapphi

In [7]:
df2

,subjectLabel,predicateLabel,objectLabel,subject,object,source
0,pearl,instance of,spherical body,Q43436,Q112511193,Q83437
1,diamond,made from material,carbon,Q5283,Q623,Q83437
2,ruby,color,red,Q43088,Q3142,Q83437
3,pearl,made from material,nacre,Q43436,Q215865,Q83437
5,ruby,streak color,white,Q43088,Q23444,Q83437
...,...,...,...,...,...,...
2915,edgrewite,space group,space group P2₁/b,Q17276566,Q102055823,Q429635
2916,poldervaartite,type locality (geology),Wessels mine,Q3907216,Q16999453,Q429635
2917,paranatisite,type locality (geology),Mount Rasvumchorr,Q3895239,Q23839192,Q429635
2918,paranatisite,type locality (geology),Material'naya adit,Q3895239,Q24036509,Q429635


In [8]:
df3 = df2[['subject','object']].melt()[['value']].drop_duplicates()
df3.columns = ['wiki_id']
df3.info()
df3.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1372 entries, 0 to 4880
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   wiki_id  1372 non-null   object
dtypes: object(1)
memory usage: 21.4+ KB


,wiki_id
0,Q43436
1,Q5283
2,Q43088
5,Q43513
13,Q138979


In [9]:
df3.to_csv('e.csv', index=False)